今までのlightBGMではスコアは頭打ち状態なので、他のアルゴリズムも試してみる。それらのアンサンブルでスコアの向上を狙ってみる。

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error

訓練データの読み込み

In [21]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange_scores = pd.read_csv('room_arrange_scores.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')

rent = pd.read_csv('rent.csv')

テストデータ

In [22]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange_scores = pd.read_csv('test_room_arrange_scores.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')

In [47]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'rf',#ランダムフォレストを使う
    'metrics':'rmse',
    'bagging_freq': 10,
    'bagging_fraction': 0.5,
    'reg_lambda': 3.681194978110037e-06,
    'max_bin': 522,
    'num_leaves': 124
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2444
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35472.1	valid_1's rmse: 41641.7
[20]	training's rmse: 32771.7	valid_1's rmse: 39412.2
[30]	training's rmse: 31925.3	valid_1's rmse: 38847
[40]	training's rmse: 31901.6	valid_1's rmse: 39228.9
Early stopping, best iteration is:
[30]	training's rmse: 31925.3	valid_1's rmse: 38847


あまり芳しくない値なので、パラメータチューニングを行ってみる。

In [61]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'rf',
        'metrics':'rmse',
        'learning_rate':0.05,
        'bagging_freq':1,
        'bagging_fraction': trial.suggest_float('reg_lambda', 0.21, 0.99, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        #verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [ ]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=200)
study.best_params

このチューニングされたパラメータを使う。

In [57]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'rf',
    'metrics':'rmse',
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'reg_lambda': 0.8541733691883097,
    'max_bin': 510,
    'num_leaves': 119
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2420
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 29013.2	valid_1's rmse: 36036.6
Early stopping, best iteration is:
[8]	training's rmse: 29070.4	valid_1's rmse: 35940.1


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

ランダムフォレストではうまくいかなそう。他のアルゴリズムを使う。

dartを使う。

In [63]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'dart',
    'metrics':'rmse',
    'lambda_l1':0.000001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1877
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 40839.2	valid_1's rmse: 45284.8
[20]	training's rmse: 26245.4	valid_1's rmse: 33594.9
[30]	training's rmse: 20647.8	valid_1's rmse: 29474.7
[40]	training's rmse: 19195.2	valid_1's rmse: 28396.9
[50]	training's rmse: 23317.3	valid_1's rmse: 31380.5
[60]	training's rmse: 27004.9	valid_1's rmse: 34222.3
[70]	training's rmse: 25752.1	valid_1's rmse: 33259.3
[80]	training's rmse: 21192.3	valid_1's rmse: 29947.9
[90]	training's rmse: 21533.4	valid_1's rmse: 30263.4
[100]	training's rmse: 22276.4	valid_1's rmse: 30863.2
[110]	training's rmse: 28096	valid_1's rmse: 

dartはよさそう。チューニングしてみる。

In [65]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'dart',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [66]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=50)
study.best_params

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2404
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52663.9	valid_1's rmse: 55365.7
[20]	training's rmse: 39657.6	valid_1's rmse: 44316.7
[30]	training's rmse: 31838.6	valid_1's rmse: 38205.8
[40]	training's rmse: 28519.3	valid_1's rmse: 35667.6
[50]	training's rmse: 31848.8	valid_1's rmse: 38332.3
[60]	training's rmse: 35992.3	valid_1's rmse: 41896.1
[70]	training's rmse: 37909.6	valid_1's rmse: 43613
[80]	training's rmse: 32864.7	valid_1's rmse: 39352.6
[90]	training's rmse: 32710.1	valid_1's rmse: 39197.6
[100]	training's rmse: 34820.4	valid_1's rmse: 41039.9
[110]	training's rmse: 41580.9	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2174
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52622.8	valid_1's rmse: 55414.6
[20]	training's rmse: 39627	valid_1's rmse: 44482.8
[30]	training's rmse: 31876.7	valid_1's rmse: 38415.4
[40]	training's rmse: 28477.7	valid_1's rmse: 35814.1
[50]	training's rmse: 31836.2	valid_1's rmse: 38484.5
[60]	training's rmse: 35907.7	valid_1's rmse: 41994.5
[70]	training's rmse: 37924.2	valid_1's rmse: 43747.6
[80]	training's rmse: 32918.6	valid_1's rmse: 39553.6
[90]	training's rmse: 32791.6	valid_1's rmse: 39384.1
[100]	training's rmse: 34829.3	valid_1's rmse: 41121.4
[110]	training's rmse: 41619.8	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000761 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2308
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52336	valid_1's rmse: 55127.3
[20]	training's rmse: 39409.2	valid_1's rmse: 44293.2
[30]	training's rmse: 31573	valid_1's rmse: 38155.4
[40]	training's rmse: 28086.9	valid_1's rmse: 35536.4
[50]	training's rmse: 31445.4	valid_1's rmse: 38130.5
[60]	training's rmse: 35695.5	valid_1's rmse: 41693.2
[70]	training's rmse: 37677.2	valid_1's rmse: 43387.8
[80]	training's rmse: 32693.4	valid_1's rmse: 39223
[90]	training's rmse: 32659.7	valid_1's rmse: 39134.5
[100]	training's rmse: 34691.5	valid_1's rmse: 40899.7
[110]	training's rmse: 41471.8	valid_1's rmse: 4667

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2370
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 53325	valid_1's rmse: 55920.6
[20]	training's rmse: 40038.4	valid_1's rmse: 44726.1
[30]	training's rmse: 32673.9	valid_1's rmse: 38847.4
[40]	training's rmse: 29628.3	valid_1's rmse: 36551.3
[50]	training's rmse: 32769.6	valid_1's rmse: 39019.7
[60]	training's rmse: 36727.5	valid_1's rmse: 42396.7
[70]	training's rmse: 38577	valid_1's rmse: 44050.6
[80]	training's rmse: 33678.6	valid_1's rmse: 39872.4
[90]	training's rmse: 33507.7	valid_1's rmse: 39686.2
[100]	training's rmse: 35475.4	valid_1's rmse: 41384.7
[110]	training's rmse: 42126.7	valid_1's rmse: 47

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001148 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2077
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 53080.9	valid_1's rmse: 55694.8
[20]	training's rmse: 39947	valid_1's rmse: 44638.4
[30]	training's rmse: 32477.7	valid_1's rmse: 38729.2
[40]	training's rmse: 29362.8	valid_1's rmse: 36385.8
[50]	training's rmse: 32500.6	valid_1's rmse: 38863.3
[60]	training's rmse: 36487.5	valid_1's rmse: 42346.8
[70]	training's rmse: 38336	valid_1's rmse: 43954.6
[80]	training's rmse: 33377.6	valid_1's rmse: 39769.6
[90]	training's rmse: 33281.7	valid_1's rmse: 39681.5
[100]	training's rmse: 35221.6	valid_1's rmse: 41331
[110]	training's rmse: 41954.5	valid_1's rmse: 4710

チューニングの結果を実際に使う。

In [ ]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'rf',#ランダムフォレストを使う
    'metrics':'rmse',
    'bagging_freq': 10,
    'bagging_fraction': 0.5,
    'reg_lambda': 3.681194978110037e-06,
    'max_bin': 522,
    'num_leaves': 124
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)